<a href="https://colab.research.google.com/github/elliottbonal/Z3metaphysicstests/blob/main/EighthTheorem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install z3-solver
from z3 import *

# Setting up the theorem solver
s = Solver()

# Defining the domain of all valid types (T143)
T143 = DeclareSort('T143')
x = Const('x', T143)

# Set-up for a reduced model of the Zmatrix being 5x5
N = 5
M = 5
Index = IntSort()

# Define the Zmatrix, Z+, predicates and the zero matrix
P = Function('P', Index, Index, T143, RealSort())
Z = Function('Z', T143, ArraySort(Index, ArraySort(Index, RealSort())))
Zplus = Function('Zplus', T143, RealSort())
zero_matrix = K(Index, K(Index, RealVal(0)))

# Declare the if Z+(x) = cardinality of set of all predicates, then all entries in the matrix are equal to 1, as per Type 2
P_cardinality = N * M
implication = Implies(
    Zplus(x) == P_cardinality,
    And([P(i, j, x) == 1 for i in range(1, N + 1) for j in range(1, M + 1)])
)
s.add(implication)

# Build matrix Z(x): Start with a Matrix full of zeros, and fill in each entry with predicate functions
Zx = K(Index, K(Index, RealVal(0)))
for i in range(1, N + 1):
    row = K(Index, RealVal(0))
    for j in range(1, M + 1):
        row = Store(row, j, P(i, j, x))
    Zx = Store(Zx, i, row)
s.add(Z(x) == Zx)

# Declare a P_2.k such that P_2.K(x) iff not P_1.1(x), as per Type 2
k = Int('k')
s.add(And(k >= 1, k <= M))
s.add(Implies(P(2, k, x) == 1,  P(1, 1, x) == 0))

# Encode previous theorem: that P_11 = 0 implies the Z matrix collapses (all entries become 0)
s.add(Implies(P(1, 1, x) == 0, Z(x) == zero_matrix))

# Trying to falsify the claim that if Z+(x) = cardinality of set of all predicates, then it collapes into nothing (Zplus(x) = 0)
s.push()
s.add(Not(Implies(Zplus(x) == P_cardinality, Zplus(x) == 0)))
result = s.check()

if result == unsat:
    print("No counterexample possible, the theorem holds.")
else:
    print("The theorem fails, counterexample found.")
    print("Model showing inconsistency:")
    print(s.model())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 47.8 MB/s eta 0:00:00
No counterexample possible, the theorem holds.
